### Process Caucasus DZ Data ###
This notebook compiles raw Caucasus DZ data from prior work stored in this repository and creates files used by `figs.ipynb`, `hf.ipynb`, `supp.ipynb`, and `supp_tables.ipynb`. All cells below should be run first prior to using those notebooks.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from geoscripts.dz import dz

This cell reads in data from the Metadata table and creates DZ sample objects from prior publications

In [ ]:
# Import Data and Create DZ Objects
metadata = pd.read_excel('Metadata-Table-Caucasus.xlsx')

names = metadata['Sample Name']
sources = metadata['Source']
latitudes = metadata['Latitude (°N)']
longitudes = metadata['Longitude (°E)']
rep_ages = metadata['Reported Age']
sample_type = metadata['Type']

color_dict = {'Modern':'red','Bedrock':'blue'}

samples = [
    dz.DZSample(name,source=sources[x],latlon=(latitudes[x],longitudes[x]),
                reported_age=rep_ages[x],color=color_dict[sample_type[x]]) 
    for x,name in enumerate(names)
    ]

This cell loads in the age data from each samples' individual Excel file and adds it to the DZ object. It also plots KDEs for each sample

In [ ]:
# Populate Objects with Age Data

paths = [sources[x] + '/' + name + '.xlsx' for x,name in enumerate(names)]

for x,sample in enumerate(samples):
    sample.agedata = pd.read_excel(paths[x])
    
    if '206/207 Pb Age' not in sample.agedata:
        sample.agedata['206/207 Pb Age'] = np.nan
    
    sample.calc_bestage('238/206 U-Pb Age','206/207 Pb Age',disc_cutoff=20,disc_age_cutoff=600)
    
    sample.kde_img()
    plt.close()
    
    sample.export_ages()
    sample.save()

In [ ]:
# Redo MDAs for Pz samples from Cowgill16 and Vasey20.
conc_dir = 'data_processing/isoplotr_rel_age_wetherill'
os.makedirs('data_processing/isoplotr_oldsamples_ages/',exist_ok=True)
os.makedirs(conc_dir,exist_ok=True)

syst_238 = {'NW-GC':0.9,'N1':1.0,'N2':0.8,'N3':0.8,'N5':0.9}

for smp in samples:
    if smp.name in ['NW-GC','N1','N2','N3','N5']:
        smp.calc_bestage(col_238='238/206 U-Pb Age',col_207='206/207 Pb Age',err_238='238 Error',err_207='206 Error',use_err=True,err_lev='1sig',
        disc_cutoff=20,disc_age_cutoff=600)
        smp.convert_1sigto2sig()
        smp.syst_238 = syst_238[smp.name]

        smp.age_238 = smp.agedata['238/206 U-Pb Age'][smp.bestage.index]
        smp.age_235 = smp.agedata['235/207 U-Pb Age'][smp.bestage.index]

        smp.agedata['75_2s'] = smp.agedata['75_1s']*2
        smp.agedata['68_2s'] = smp.agedata['68_1s']*2

        smp.calc_mda(systematic=True,filter235238=True,cutoffs235238=(80,110))
        smp.calc_ysg(systematic=True,filter235238=False,cutoffs235238=(80,110))

        # Output ratios for concordia plots
        grain_min = np.min(smp.mda_ages.index)
        grain_max = np.max(smp.mda_ages.index)

        input_cols = ['75_ratio','75_2s','68_ratio','68_2s','rho']
        cols = ['207Pb/235U','207Pb/235U_err','206Pb/238U','206Pb/238U_err','error_corr']

        rename_dict = {cols[x]:input_cols[x] for x in range(len(cols))}
        
        for col in cols:
            smp.agedata[col] = smp.agedata[rename_dict[col]]

        smp.agedata['Best age'] = smp.bestage

        agedata_sorted = smp.agedata.sort_values(by=['Best age'],ascending=True,ignore_index=True)

        output_ygc = agedata_sorted.loc[grain_min:grain_max,cols]
        output_ygc.to_csv(conc_dir + '/' + smp.name +'_ygcratios.csv',index=False)
        grain_ysg = agedata_sorted[agedata_sorted['Best age']==smp.ysg].index[0]
        
        print(smp.name,grain_ysg,smp.ysg)
        output_ysg = pd.DataFrame([agedata_sorted.loc[grain_ysg,cols]],columns=cols)
        output_ysg.to_csv(conc_dir + '/' + smp.name +'_ysgratios.csv',index=False)
        
        
        # Output 235/238 concordant grains for MLA
        conc238235 = np.round((smp.age_238/smp.age_235*100),0)
        accept = (conc238235>=80) & (conc238235<=110) 

        iso_ages = pd.DataFrame([],columns=['tbest','err'])
        iso_ages['tbest'] = smp.bestage[accept]
        
        # Convert 2s errors to 1s
        iso_ages['err'] = smp.besterror[accept]/2

        iso_ages.to_csv('data_processing/isoplotr_oldsamples_ages/' + smp.name + '.csv',index=False)

        smp.save()

This cell saves the data in the DZ objects to a SHP file for GIS use.

In [ ]:
#%% Save to Shapefile

gdf = dz.write_file(samples,'caucasus_dz.shp')

This cell exports modern river samples to a separate csv for catchment analysis.

In [ ]:
names = []
lats = []
lons = []
for sample in samples:
    if sample.color=='red':
        names.append(sample.name)
        lats.append(sample.latlon[0])
        lons.append(sample.latlon[1])

data = {'Modern Sample':names,'Latitude':lats,'Longitude':lons}
df = pd.DataFrame(data)
df.to_csv('modern_samples.csv')